In [ ]:
import pandas as pd

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = "plotly"
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# Load experiment
def get_experiments(difficult_location_number: int = 1, n_days: int = 10):
    """
    Returns a list of dicts, with each dict being an experiment with:
    - name
    - dataset
    - test_length # length counted from the end of the data that will be considered the Test period
    -"""
    complete_data = pd.read_csv(
        f"data/model_input_difficult_location_{difficult_location_number}.csv",
        parse_dates=True,
        index_col=0,
    )[:-3]

    def create_dataset(i: int):
        dataset = complete_data.iloc[0 * 24 * 4: (120 + i) * 24 * 4 + 24 * 4]
        dataset = dataset[~dataset["load"].isna()]
        return dataset

    # Generate the experiments where we forecast for 1 day each time
    # Dataset contains both the 120 days of training data + the 1 day of test data
    result = [
        dict(  # Predict day 1
            name=f"N{i + 1}_loc{difficult_location_number}",
            dataset=create_dataset(i),
            test_length=24 * 4 if i != n_days - 1 else 24 * 4 - 3,
        )
        for i in range(n_days)
    ]
    return result


In [ ]:
experiments = get_experiments(1)

experiment = experiments[0]

In [ ]:
import openstef.feature_engineering.apply_features

# df_dataset = experiment["dataset"].copy()
df_dataset = openstef.feature_engineering.apply_features.apply_features(
    experiment["dataset"].copy()
)

df_dataset = df_dataset.dropna()
df_dataset = df_dataset * 1.0
df_dataset = df_dataset.resample("15min").fillna("backfill")
df_dataset

In [ ]:
# Prepare data
df_train = df_dataset.iloc[: -experiment["test_length"]]
df_test = df_dataset.iloc[-experiment["test_length"]:]

In [ ]:
import scipy.fft as sf

load = df_train['load'].values
N = load.shape[0]
X = sf.rfft(load) / N
freqs = sf.rfftfreq(n=N, d=1/12)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Define parameters
measurement_interval_minutes = 15
# window_size_days = 7
window_size_days = 7

# Calculate window size
window_size_minutes = window_size_days * 24 * 60
window_size_samples = int(window_size_minutes / measurement_interval_minutes)
N = window_size_samples

W_start = 0
W_end = W_start + N

# Generate frequency list using rfftfreq
freqs = np.fft.rfftfreq(N, d=1.0)

# Print the frequency list
print("Frequency List:")
print(f'Shape: {freqs.shape}')

In [ ]:
X = sf.rfft(load[W_start:W_end]) / N
print(f'Shape: {X.shape}')

In [ ]:
# Plot DFT
ax = plt.stem(freqs[:100], abs(X[:100]))

In [ ]:
plt.plot(sf.irfft(X))

In [ ]:
X_new = np.copy(X)
X_new[abs(X) < 0.4*1e6] = 0

plt.stem(freqs[:100], abs(X_new[:100]))
plt.show()
Y = sf.irfft(X_new, n=N)
Y = (Y / abs(Y).max()) * load[W_start:W_end].max()

plt.plot(Y)
plt.plot(load[W_start:W_end])
plt.show()

In [ ]:
def fourierExtrapolation(X, n, n_predict):
    f = np.fft.rfftfreq(n, d=1.0)              # frequencies
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in range(len(X)):
        ampli = np.absolute(X[i]) / n   # amplitude
        phase = np.angle(X[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig

In [ ]:
forecast_N = 192 # 2 days
preview_N = 672 # 7 days

Y = fourierExtrapolation(X_new, N, forecast_N)
Y = (Y / abs(Y).max()) * load[W_start:W_end].max()

plt.plot(Y[-(preview_N + forecast_N):])
plt.plot(load[W_end - preview_N:W_end + forecast_N])
# Add a vertical line
plt.axvline(x=preview_N, color='r', linestyle='--')

plt.show()

In [ ]:
N

In [ ]:
from typing import List
import scipy.fft as sf
import numpy as np

def fourier_forecast(
        signal: np.ndarray,
        top_K: int = 5,
        forecast_N: int = 192,
):
    N = signal.shape[0]
    X = sf.rfft(signal) / N
    X_thresholded = np.copy(X)
    top_indices = np.argsort(abs(X_thresholded))[::-1][top_K:]
    X_thresholded[top_indices] = 0
    
    # Generate restored signal
    frequencies = np.fft.rfftfreq(N, d=1.0)
    t = np.arange(0, N + forecast_N)
    signal_restored = np.zeros(t.size)
    for i in range(len(X_thresholded)):
        amplitude = np.absolute(X_thresholded[i]) / N
        phase = np.angle(X_thresholded[i])
        signal_restored += amplitude * np.cos(2 * np.pi * frequencies[i] * t + phase)
    
    # Normalize the signal
    signal_restored = (signal_restored / abs(signal_restored).max()) * signal.max()
    
    return signal_restored

# The following function adds a fourier forecast feature to the dataset
# It takes an arbitrary long signal, a window size and forecast horizons
# For each point in the signal, it calculates the fourier forecast using the previous window

def fourier_forecast_feature(
        signal: np.ndarray,
        window_size: int,
        horizons: List[int],
        top_K: int = 5,
):
    N = signal.shape[0]
    forecast_features = np.zeros((N, len(horizons)))
    forecast_features[:, :] = np.nan
    
    for i in range(N):
        if i < window_size:
            continue
            
        signal_window = signal[i - window_size+1:i+1] # +1 to include the current point
        forecast = fourier_forecast(signal_window, top_K=top_K, forecast_N=max(horizons) + 1)[window_size:]
        
        for j, horizon in enumerate(horizons):
            forecast_features[i, j] = forecast[horizon]
            
    return forecast_features


In [ ]:
# Define parameters
measurement_interval_minutes = 15
# window_size_days = 7
window_size_days = 7

# Calculate window size
window_size_minutes = window_size_days * 24 * 60
window_size_samples = int(window_size_minutes / measurement_interval_minutes)
N = window_size_samples

forecast_N = 192 # 2 days

signal = load[0:N]
Y = fourier_forecast(signal, top_K=5, forecast_N=forecast_N)

plt.plot(Y[-(forecast_N + N):])
plt.plot(load[0:N + forecast_N])
# Add a vertical line
plt.axvline(x=N, color='r', linestyle='--')

plt.show()

In [ ]:
features = fourier_forecast_feature(
    load[:2000],
    window_size=window_size_samples,
    horizons=[0, 96, 192],
    top_K=10,
)

In [ ]:
plt.plot(load[window_size_samples:2000])
plt.plot(features[window_size_samples:, 0]) # 15 min ahead
plt.plot(features[window_size_samples:, 1])
plt.plot(features[window_size_samples:, 2]) # 2 days ahead